# Setup

In [ ]:
%matplotlib inline
from pylab import *
import math
import numpy as np

import detectorgeometry_trap as geom_trap
reload(geom_trap)

import decay_properties as decay
reload(decay)

import trigger_properties as RF
reload(RF)

## Creation of Graph with Successive Effective Area Refinement

In [ ]:
phi_src = radians(180)
h = 35.5 #km
R = 6356. # km
ice =4
theta_view = radians(1) #degrees
N = 10000000 # number of samples 1000000
E_nu = 18 #log energy in eV of incident neutrino 
E_cut = 14
f_lo = 180
f_high = 1200
Gain_dB = 10
Nphased = 1
Epk_to_pk_threshold = 284e-6

In [ ]:
A_g = []
A_exit=[]
A_exit_energy=[]
A_exit_energy_decay=[]
A_rf = []

dist_med=[]
eng_med=[]


# Creating a plot for the full range of possible source declinations 
start = 50 # minimum source declination in degrees
end = 80 # maximum source declination in degrees
steps=60 # divisions indeclination used to generate plot
x1 = np.linspace(-(90-start),-(90-end),steps)

for j in range(0,steps):
    theta_src = radians(start+(end-start)*float(j)/steps)  
    geom_area = geom_trap.Area(theta_src, h,R,theta_view, N)
    A_deg, phi_e, theta_e, view_angle, exit_angle, emg_angle, norm, e_dot, A0, N0 = geom_area.event_retention()
    decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, \
                             norm,E_nu, h, R, theta_view, exit_angle, emg_angle, view_angle, A_deg, A0, N0,0,0)
    A_1,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_emg, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit, A0, N0 = decay1.degree_eff_area()
    decay3 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, \
                             norm,E_nu, h, R, theta_view, exit_angle, emg_angle, view_angle, A_deg, A0, N0,1,1)

    A_3,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_emg, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit, A0, N0 = decay3.degree_eff_area()
    A_trigger =0
    if len(ret_e_dot) != 0:
        trigger_result = RF.event_detection(A_3, f_lo, f_high, Gain_dB, Nphased, \
                                        ret_exit,ret_emg,ret_view, ret_decay_angle, ret_decay_alt, \
                                        theta_view, ret_exit_obs,ret_exit_decay, ret_decay_obs, ret_p_exit, ret_e_dot,\
                                        ret_tau_energy, R, ice, h, ret_t_e, ret_phi_e,theta_src, Epk_to_pk_threshold, A0,N0)
        A_trigger = trigger_result.RF_eff_area()

    
    A_g.append(A_deg)
    A_exit.append(A_1)
    A_exit_energy_decay.append(A_3)
    A_rf.append(A_trigger)
    print round(degrees(theta_src))
    
start = 80.5 # minimum source declination in degrees
end = 90 # maximum source declination in degrees
steps=100 # divisions indeclination used to generate plot
x2 = np.linspace(-(90-start),-(90-end),steps)

for j in range(0,steps):
    theta_src = radians(start+(end-start)*float(j)/steps)  
    geom_area = geom_trap.Area(theta_src, h,R,theta_view, N)
    A_deg, phi_e, theta_e, view_angle, exit_angle, emg_angle, norm, e_dot, A0, N0 = geom_area.event_retention()
    decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, \
                             norm,E_nu, h, R, theta_view, exit_angle, emg_angle, view_angle, A_deg, A0, N0,0,0)
    A_1,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_emg, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit, A0, N0 = decay1.degree_eff_area()
    decay3 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, \
                             norm,E_nu, h, R, theta_view, exit_angle, emg_angle, view_angle, A_deg, A0, N0,1,1)

    A_3,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_emg, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit, A0, N0 = decay3.degree_eff_area()
    A_trigger =0
    if len(ret_e_dot) != 0:
        trigger_result = RF.event_detection(A_3, f_lo, f_high, Gain_dB, Nphased, \
                                        ret_exit,ret_emg,ret_view, ret_decay_angle, ret_decay_alt, \
                                        theta_view, ret_exit_obs,ret_exit_decay, ret_decay_obs, ret_p_exit, ret_e_dot,\
                                        ret_tau_energy, R, ice, h, ret_t_e, ret_phi_e,theta_src, Epk_to_pk_threshold, A0,N0)
        A_trigger = trigger_result.RF_eff_area()

    
    A_g.append(A_deg)
    A_exit.append(A_1)
    A_exit_energy_decay.append(A_3)
    A_rf.append(A_trigger)
    print round(degrees(theta_src))


In [ ]:
fig = plt.figure(figsize=(7,7))
ax = plt.axes(yscale='log')
x_full = np.concatenate((x1,x2))

ax.plot(x_full, A_g)
ax.plot(x_full, A_exit)
ax.plot(x_full, A_exit_energy_decay)
ax.plot(x_full, A_rf)

#plt.fill_between(x2, sky_maps[0],sky_maps[1],color='tab:gray')
horizon = degrees(np.arcsin(R / (R+h)))-90
plt.xlim(-40,0)  
plt.ylim(10**-11,3*10**2)  
legend = ax.legend([r'$\langle A \rangle_g$', 
                    r'$\langle A \rangle_g \cdot P_{exit}$', 
                    r'$\langle A \rangle_g \cdot P_{exit} \cdot P_{shower} \cdot P_{decay}$',
                   r'$\langle A \rangle_g \cdot P_{exit} \cdot P_{shower}\cdot P_{decay} \cdot P_{trigger} $'],
                   fontsize=14, bbox_to_anchor=(0.8, -0.15)) #
plt.setp(legend.get_title(),fontsize='x-large')

plt.xlabel("Payload Elevation Angle of Source "+r'$\theta_{ele}$'+" (degrees)", fontsize=18)
plt.ylabel(r'$[km^2]$', fontsize=18)
plt.title("ANITA "+r'$10^{18}$'+" eV Effective Area", fontsize=20)




plt.text(-7.5,8*10**-8,'horizon',rotation=90,fontsize=12)
plt.text(-29,3*10**-7,'A1',rotation=90,fontsize=12)
plt.text(-36.5,3*10**-7,'A3',rotation=90,fontsize=12)
horizon = degrees(np.arcsin(R / (R+h)))-90
A1 = -27.4
A3 = -35
plt.axvline(x=horizon,color='r')
plt.axvline(x=A1,color='k')
plt.axvline(x=A3,color='k')

ax.tick_params(axis = 'both', which = 'major', labelsize = 16)

y_min =min(A_g)
y_max = max(A_g)
print y_min, y_max
y_ticks = [10**x for x in range(int(round(math.log10(10**-11))),int(round(math.log10(y_max)))+1)]
plt.yticks(y_ticks)
ax.grid(True,which='both')
#plt.semilogy()
plt.show()